# 1.Import packages

In [10]:
# Importing all required packages at the start of the notebook
import IPython

from qiime2 import Visualization

import pandas as pd

# 2.Import the data

In [4]:
# Location
data_dir = "Project_data/Diversity"
! mkdir -p "$data_dir"

# 3.Determination of the sampling depth

In [3]:
! qiime feature-table summarize \
    --i-table Project_data/Taxonomy/table_filtered.qza \
    --m-sample-metadata-file Project_data/Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/table_filtered.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/table_filtered.qzv


In [4]:
Visualization.load(f"{data_dir}/table_filtered.qzv")

<visualization: Visualization uuid: 82d0c2dc-623c-47d3-8d81-0032c7a5785c>

In [5]:
! qiime diversity alpha-rarefaction \
    --i-table Project_data/Taxonomy/table_filtered.qza \
    --p-max-depth 80000 \
    --m-metadata-file Project_data/Metadata/updated_fungut_metadata.tsv \
    --o-visualization $data_dir/alpha-rarefaction.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/alpha-rarefaction.qzv


In [6]:
Visualization.load(f"{data_dir}/alpha-rarefaction.qzv")

<visualization: Visualization uuid: 1c3dba73-2629-436e-9d79-c251ac8e0007>

According to alpha rarefication, a sampling depth of 20.000 was chosen, since the Shannon and observed feature metrics start to plateau at this point. Referring to the feature table for this sampling depth results in a retention of 2.720.000 reads (40.81%) across 136 samples (90.67%).

# 4.Euler
The diversity analysis was performed using the `q2-boots` plugin for QIIME2. To run the bootstrapping with a sufficiently high number of iterations (`n = 1000`), this step was performed on Euler. As this plugin was not included in the previously installed MOSHPIT distribution, the Amplicon distribution had to be installed additionally via Miniconda.

## 4.1 Import files
As with the 2.Taxonomy script, the files required to run the bootstrapping on Euler were uploaded to Polybox for download by the script running on Euler.

## 4.2 Bootstraping script
The following script was run on Euler.

```bash
#!/bin/bash
#SBATCH --job-name=bootstraping
#SBATCH --time=24:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=32G
#SBATCH --output=bootstraping_%j.out
#SBATCH --error=bootstraping_%j.err
#SBATCH --mail-type=END,FAIL

# Activate conda
source ~/miniconda3/etc/profile.d/conda.sh
conda activate qiime2-amplicon-2025.10

# Data folder
data_dir="ProjectData"


# Download the meta data and reads
module load eth_proxy

wget --content-disposition -nc --progress=dot:giga -P "$data_dir" https://polybox.ethz.ch/index.php/s/e7ieANgiAn26oBs/download
wget --content-disposition -nc --progress=dot:giga -P "$data_dir" https://polybox.ethz.ch/index.php/s/xNSLKnR2y3QG9eb/download
wget --content-disposition -nc --progress=dot:giga -P "$data_dir" https://polybox.ethz.ch/index.php/s/KscLWzSGnkmEmY5/download

echo "Download done!"

# Run the bootstraping
qiime boots kmer-diversity \
  --i-table $data_dir/table_filtered.qza \
  --i-sequences $data_dir/rep-seqs_filtered.qza \
  --m-metadata-file $data_dir/updated_fungut_metadata.tsv\
  --p-sampling-depth 20000 \
  --p-n 1000 \
  --p-replacement \
  --p-alpha-average-method median \
  --p-beta-average-method medoid \
  --output-dir $data_dir/boots-kmer-diversity

echo "Bootstraping done!"
```

# 5.Diversity
The files created by the script on Euler were downloaded and uploaded to Polybox in order to be accessible for this script.

In [5]:
%%bash -s $data_dir

wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/nmb4j2YDSJbjJP2/download
wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/sYGkqwCffpcK8Si/download
wget --content-disposition -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/XJFWGkkNYfZSyse/download

chmod -R +rxw "$1"

--2025-11-18 13:01:38--  https://polybox.ethz.ch/index.php/s/nmb4j2YDSJbjJP2/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
--2025-11-18 13:01:38--  https://polybox.ethz.ch/index.php/s/sYGkqwCffpcK8Si/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
--2025-11-18 13:01:38--  https://polybox.ethz.ch/index.php/s/XJFWGkkNYfZSyse/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81537205 (78M) [application/octet-stream]
Saving to: ‘Project_data/Diversity/braycurtis.qza’

     0K ........ ........ ........ ........ 41%  418M 0s
 32768K ..

## 5.1 Alpha diversity

In [6]:
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/shannon.qza \
  --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
  --o-visualization $data_dir/alpha_group_significance.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/alpha_group_significance.qzv


In [7]:
Visualization.load(f"{data_dir}/alpha_group_significance.qzv")

<visualization: Visualization uuid: 705f2693-2a79-483b-9ee9-fb99abec4fbf>

## 5.2 Beta diversity

In [8]:
Visualization.load(f"{data_dir}/scatter_plot.qzv")

<visualization: Visualization uuid: 23878c3e-47b1-40ee-8355-a55864d550d7>

In [ ]:
meta_data_df = pd.read_csv(f"{data_dir}/../Metadata/updated_fungut_metadata.tsv", sep="\t")

In [28]:
meta_data_df.columns

Index(['ID', 'country_sample', 'state_sample', 'latitude_sample',
       'longitude_sample', 'sex_sample', 'age_years_sample',
       'height_cm_sample', 'weight_kg_sample', 'bmi_sample',
       'diet_type_sample', 'ibd_sample', 'gluten_sample', 'age_range',
       'bmi_category', 'continent'],
      dtype='object')

In [31]:
meta_cols = ["age_range", "country_sample", "sex_sample", "bmi_sample", "diet_type_sample", "ibd_sample", "gluten_sample", "continent"]

for col in meta_cols:
    output_name = f"{data_dir}/bray_curtis-{col}-significance.qzv"
    print(f"Running for column: {col}")

    ! qiime diversity beta-group-significance \
        --i-distance-matrix $data_dir/braycurtis.qza \
        --m-metadata-file $data_dir/../Metadata/updated_fungut_metadata.tsv \
        --m-metadata-column {col} \
        --p-pairwise \
        --o-visualization {output_name}
    
# Errors with country_sample (only unique values), bmi_sample (numeric type) 

Running for column: age_range
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: Project_data/Diversity/bray_curtis-age_range-significance.qzv
Running for column: country_sample
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Plugin error from diversity:

  All values in the grouping vector are unique. This method cannot operate on a grouping vector with only unique values (e.g., there are no 'within' distances beca

In [32]:
Visualization.load(f"{data_dir}/bray_curtis-age_range-significance.qzv")

<visualization: Visualization uuid: 38479e0c-1f9a-4187-a38a-dc0cf8919e50>

In [34]:
Visualization.load(f"{data_dir}/bray_curtis-diet_type_sample-significance.qzv")

<visualization: Visualization uuid: 431caea2-0541-4404-a409-d01a85c1e46a>

In [35]:
Visualization.load(f"{data_dir}/bray_curtis-ibd_sample-significance.qzv")

<visualization: Visualization uuid: fcfd6096-4ee0-4a1e-861e-8458027c0323>

In [36]:
Visualization.load(f"{data_dir}/bray_curtis-sex_sample-significance.qzv")

<visualization: Visualization uuid: 19fa9f6b-7100-434e-b113-9d0c91562cc2>

In [37]:
Visualization.load(f"{data_dir}/bray_curtis-continent-significance.qzv")

<visualization: Visualization uuid: 8b4c7826-e770-41cb-8ed8-f5e3836990fc>